In [1]:
#| default_exp regress_ml

# Train ML model

In [2]:
#|hide
from nbdev.showdoc import *

In [6]:
#|hide
#|export
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from ML_projects import const_vals as CONST

In [16]:
#|export
#TODO - take out the randorm forest and put it as separate function 
class TrainRegression():
       
       def __init__(self,
               df_path : str , # path to dataframe to be used to train. File should be CSV file
               ground_truth_col: str, # name of the column with true data to train
               test_size : float , #size of data to be used for test 

               columns_to_remove : list[str]=None , #columns not to use for trainning the model. These columns will be removed.
               ):
             self.df_path = df_path
             self.columns_to_remove = columns_to_remove
             self.ground_truth_col = ground_truth_col
             self.test_size = test_size
             self.x_train, self.x_test, self.y_train, self.y_test =self._load_df_split_data()
             

             self.train_model()

       def _load_df_split_data(self):
               
               self.df = pd.read_csv(self.df_path)
               #load dataframe
               if self.columns_to_remove!= None:
                     self.df = self.df.drop(self.columns_to_remove,axis=1)
          
               # split to x,y and train test data
               self.x = self.df.drop(self.ground_truth_col,axis=1)
               self.y = self.df[self.ground_truth_col].values

               #split data to train and test
               self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(
                      self.x, self.y, test_size=self.test_size, random_state=42)

               return self.x_train, self.x_test, self.y_train, self.y_test
       

       def train_model(self):
              # create base model
              rf = RandomForestRegressor()

              rf_random = RandomizedSearchCV(estimator = rf, 
                                             param_distributions = CONST.random_grid_rf,
                                               n_iter = CONST.n_iteration_rf,
                                                 cv = CONST.cv_rf, 
                                                 verbose=CONST.VERBOSE , 
                                                 random_state=CONST.RANDOM_STATE , 
                                                 n_jobs = CONST.N_JOBS)
                      
              # Fit the random search model
              rf_random.fit(self.x_train, self.y_train)
              
              #train model 
              self.best_rf_params = rf_random.best_params_

              #fit best model
              self.rf_model = RandomForestRegressor(**self.best_rf_params)

              self.rf_model.fit(self.x_train,self.y_train)
              #predict

              y_pred = self.rf_model.predict(self.x_test)

              self.r2 = r2_score(y_pred, self.y_test)



              return self.rf_model




In [17]:
instance = TrainRegression(
  df_path=r"D:\git\ML_projects\nbs\data\resampled_sen2.csv",
  ground_truth_col = "TOC",
  test_size = 0.25,
  columns_to_remove = ['Unnamed: 0.1', 'Unnamed: 0', 'Lon', 'Lat', 'clay', 'silt','sand', 'NI']    
                    )

x_train,x_test, y_train, y_test=instance._load_df_split_data()


Fitting 3 folds for each of 100 candidates, totalling 300 fits


d:\git\ML_projects\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
153 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
92 fits failed with the following error:
Traceback (most recent call last):
  File "d:\git\ML_projects\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\git\ML_projects\.venv\Lib\site-packages\sklearn\base.py", line 1344, in wrapper
    estimator._validate_params()
  File "d:\git\ML_projects\.venv\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "d:\git\ML_projects\.venv\Lib\site-package

In [18]:
instance.r2

-0.5664377102446359

In [ ]:
y_test